In [1]:
from model_lstm import *
from util_lstm import *
from sklearn.preprocessing import LabelEncoder
#l = 'awake,diaper,hug,hungry,sleepy,uncomfortable'.split(',')
le = LabelEncoder()
#l_ft = le.fit_transform(l)
from keras.utils import to_categorical

if 1:
    te = get_test_data()
    tr = get_train_data()
    

/home/wb/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/wb/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wb/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wb/anaconda3/lib/python3.6/site-packages/tensorflow/p

../input/test/test_207.f0.csv


 38%|███▊      | 87/228 [00:14<00:22,  6.24it/s]

../input/test/test_225.f0.csv


 71%|███████▏  | 163/228 [00:28<00:11,  5.80it/s]

...

 86%|████████▌ | 196/228 [00:33<00:05,  6.35it/s]

../input/test/test_47.f0.csv


 89%|████████▉ | 203/228 [00:34<00:04,  6.15it/s]

../input/test/test_61.f0.csv


100%|██████████| 913/913 [02:42<00:00,  5.64it/s]


In [13]:
    X_test = te[0]

    a = X_test[0]
    for b in X_test[1:]:
        B = b.copy()
        #print(B.shape)
        if B.shape[0] == 0:
            B = X_test[0].copy()
        c = np.concatenate([a,B])
        a = c.copy()
    X_test = a.copy()
    y_test_f = te[1]


In [48]:
    a=tr[0][0]
    for b in tqdm(tr[1:]):
        B = b[0].copy()
        #print(B.shape)
        if B.shape[0] == 0:
            B = tr[0][0].copy()
        c = np.concatenate([a,B])
        a = c.copy()
    X_train = a.copy()
    

100%|██████████| 889/889 [00:05<00:00, 176.30it/s] 


In [50]:
    y_train_f = [i[1] for i in tr]

In [54]:
labels = []
for e in y_train_f:
    for j in e:
        labels.append(j)

In [59]:
y_train_transformed = le.fit_transform(labels)
y_train = to_categorical(y_train_transformed)

In [60]:
y_train.shape

(7976, 6)

In [67]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=12,init='k-means++',n_init=10,max_iter=600,random_state=0)


In [68]:
X_train_pred = km.fit_predict(X_train)

In [92]:
clips_train = [len(a) for a in y_train_f]
feats_train = []
for clips in clips_train:
    temp = np.zeros((12,1))
    feats_train.append(temp)

In [93]:
start = 0
idx = 0
for clips in clips_train:
    end = start + clips
    
    for i in X_train_pred[start:end]:
        feats_train[idx][i] += 1
    idx += 1
        
    
    start = end


In [109]:

y_train_short = [w[0] for w in y_train_f]

In [112]:
from sklearn.preprocessing import LabelEncoder
le2 = LabelEncoder()

y_train_short_tr = le2.fit_transform(y_train_short)

In [114]:
y_train_short_tr.shape

(890,)

In [122]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(feats_train,y_train_short_tr)


In [137]:
X_TRAIN = np.array(X_TRAIN)
X_TEST = np.array(X_TEST)
Y_TRAIN = np.array(Y_TRAIN)
Y_TEST = np.array(Y_TEST)

X_TRAIN = np.array(X_TRAIN)
X_TEST = np.array(X_TEST)
Y_TRAIN = np.array(Y_TRAIN)
Y_TEST = np.array(Y_TEST)


X_TRAIN = X_TRAIN.squeeze()
Y_TRAIN = Y_TRAIN.squeeze()

X_TEST = X_TEST.squeeze()
Y_TEST = Y_TEST.squeeze()
#Y_TRAIN = np.expand_dims(Y_TRAIN,axis=-1)
#Y_TEST = np.expand_dims(Y_TEST,axis=-1)






In [138]:
xgb.fit(X_TRAIN,Y_TRAIN)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [139]:
train_preds = xgb.predict(X_TEST)

In [141]:
train_preds


array([2, 4, 4, ..., 4, 0, 2])

In [142]:
Y_TEST

array([3, 2, 0, ..., 2, 0, 5])

In [143]:
(train_preds == Y_TEST).sum() / len(Y_TEST)

0.16143497757847533